In [5]:
# if you have this file already , please put the path of it in the drive
!cp  /content/drive/MyDrive/abdo/chomicha_wajabatUrl.json wajabatUrl.json
!cp  /content/drive/MyDrive/abdo/chomicha_cat.json cat.json

In [ ]:
import requests
import bs4


ToArray = lambda name , rng : [f"https://www.choumicha.ma/c/{name.replace('.html', f'')}/Page-{str(index + 1)}.html" for index in range(rng)]
urls = []
def checkD(text: str) -> bool:

    for chr in text:
        if chr.isdigit():
            return True
    return False


def getN(text: str) -> int:
    text = text.split("/")[-1].replace(".html", "")
    n = 0
    ten = 1
    for chr in text:
        if chr.isdigit():
            n = 10 * n + int(chr)
    return n


def getFin(url: str)-> int :
    html = requests.get(url)
    soup = bs4.BeautifulSoup(html.text, "html.parser")
    ul = soup.find(class_="pagination-end")
    if ul is None:
        return None
    return  getN(ul.find("a").get("href"))


def getBaseUrls():
    html = requests.get("https://www.choumicha.ma/c/gateaux-marocains/Page-5.html")
    soup = bs4.BeautifulSoup(html.text, "html.parser")
    ul = soup.find(class_="level1")
    global  urls
    li = ul.findAll("li")
    for i in li:
        u = str(i.find("a").get("href")).split("/")[-1].replace(".html" , "")
        if checkD(u):
            urls.append(f'https://www.choumicha.ma{i.find("a").get("href")}')
        else:
            numberOfWajabat = getFin(f'https://www.choumicha.ma{i.find("a").get("href")}')
            if numberOfWajabat is None:
              urls.append(i.find("a").get("href"))
              continue
            urls += ToArray(u,numberOfWajabat)

In [ ]:
getBaseUrls()

In [ ]:
len(urls)

362

In [ ]:
for i in range(len(urls)):
  if "https"  not in urls[i]:
    urls[i] = f'https://www.choumicha.ma{urls[i]}'


In [ ]:
urls

[]

In [ ]:
import json
with open("cat.json" , "w") as f:
  json.dump(urls, f)

In [ ]:
!cp cat.json /content/drive/MyDrive/abdo/chomicha_cat.json

In [ ]:
with open("cat.json" , "r") as f:
  URLS = json.load(f)

In [ ]:
len(URLS)

362

In [ ]:
import requests
import bs4
urlBase = "https://www.choumicha.ma"
# html = requests.get("https://www.choumicha.ma/accueil-recettes/Page-14.html")
# soup = bs4.BeautifulSoup(html.text,'html.parser')
# divs = soup.findAll(class_="span4")

def getWajabatUrl(url):
    links = []
    html = requests.get(url)
    soup = bs4.BeautifulSoup(html.text, 'html.parser')
    divs = soup.findAll(class_="span4")
    for div in divs:
        link = div.find("a").get("href")
        if link is not None:
            links.append(urlBase+link)
    return links


In [ ]:
wajabatUrls = []
def saveWajabatUrl():
  global wajabatUrls
  for url in URLS:
    wajabatUrls += getWajabatUrl(url)
  with open("wajabatUrl.json" ,"w") as f:
    json.dump(wajabatUrls ,  f)

12

In [ ]:
# avec chwiya dyal l3ya9a

from tqdm import tqdm
import json
wajabatUrls = []
def saveWajabatUrl():
    global wajabatUrls

    with tqdm(total=len(URLS), desc="Processing URLs") as pbar:
        for url in URLS:
            wajabatUrls += getWajabatUrl(url)
            pbar.update(1)

    with open("wajabatUrl.json", "w") as f:
        json.dump(wajabatUrls, f)


In [ ]:
saveWajabatUrl()

Processing URLs: 100%|██████████| 362/362 [09:25<00:00,  1.56s/it]


In [ ]:
len(wajabatUrls)

3170

In [ ]:
!cp wajabatUrl.json /content/drive/MyDrive/abdo/chomicha_wajabatUrl.json

In [1]:
import requests
import bs4

urlBase = "https://www.choumicha.ma"


# html = requests.get("https://www.choumicha.ma/accueil-recettes/Page-14.html")
# soup = bs4.BeautifulSoup(html.text,'html.parser')
# divs = soup.findAll(class_="span4")
def getInfoOfWajba(url):
    html = requests.get(url)
    soup = bs4.BeautifulSoup(html.text, "html.parser")
    data = {}
    data["url"] = url
    data["title"] = soup.find("h1", class_="fn").text
    recipe_info = soup.find("ul", class_="recipe-info").findAll("li")
    lis = {}
    for li in recipe_info:
        if li.find("strong") is not None  and  li.find("span") is not None:
            lis[str(li.find("strong").text).split(":")[0]] = li.find("span").text
    data["recipe-info"] = lis
    ingredients = soup.findAll(class_="ingredient")
    ingredients = [li.find("span" , class_="name").text for li in ingredients]
    data["ingredients"] = ingredients
    prepaDiv =  soup.find(class_="preparation").findAll("li")
    preparation = []
    for li in prepaDiv:
        preparation.append(str(li.get_text()))
    data["preparation"] = preparation
    data["categories"] = [a.get_text() for a in soup.select(".recette_divider li .row-fluid .row-fluid div a") if
                          a.get_text() != "Autres"]
    data["mots cles"] = [a.get_text() for a in soup.select(".recette_divider li .clearfix span a")]
    return data

In [7]:
import json
with open("/content/wajabatUrl.json" , "r") as f:
  URLS = json.load(f)

In [8]:
len(URLS)

3170

In [10]:
wajabatInfo = []
from tqdm import tqdm

def siiiir(urls):
    global wajabatInfo
    with tqdm(total=len(urls), desc="Processing URLs") as pbar:
        for url in urls:
            try:
              wajabatInfo.append(getInfoOfWajba(url))
            except Exception as e :
              print(e)
            pbar.update(1)




In [30]:
siiiir(URLS[1558:])


Processing URLs: 100%|██████████| 1612/1612 [39:59<00:00,  1.49s/it]


In [31]:
with open("choumichaWajabat.json" , "w") as f:
  json.dump(wajabatInfo , f)

In [32]:
!cp choumichaWajabat.json /content/drive/MyDrive/abdo/chomicha_data.json

In [33]:
len(wajabatInfo)

3171

In [29]:
getInfoOfWajba("https://www.choumicha.ma/recette/1369-gratin-de-poulet-et-de-fruits-de-mer.html")

{'url': 'https://www.choumicha.ma/recette/1369-gratin-de-poulet-et-de-fruits-de-mer.html',
 'title': 'Gratin de poulet et de fruits de mer',
 'recipe-info': {'Type': 'Cuisine du monde',
  'Temps de cuisson': '10m',
  'Temps total': '40m',
  'Nombre de personnes': '4'},
 'ingredients': ['250 g de blanc de poulet émincé',
  '250 g de crevettes décortiquées',
  '250 g de calamars coupés en morceaux',
  "1 gousse d'ail émincée",
  '1 c. à soupe de persil et de coriandre émincés',
  '2 pincées de poivre',
  '1/2 c. à café de piment fort (facultatif)',
  'Sel',
  '100 g de mozzarella râpée',
  'Pour la sauce béchamel :',
  '40 g de beurre',
  '40 g de farine',
  '45 cl de bouillon de crevettes (Cliquez sur lien dans la préparation)',
  '2 pincées de poivre',
  '2 pincées de piment fort',
  'Sel',
  'Pour gratiner :',
  '80 g de mozzarella râpée'],
 'preparation': ['Sauce béchamel au bouillon de crevettes :',
  'Dans une casserole sur feu doux, faire fondre le beurre et le saupoudrer de farin